In [66]:
import search_engine
import math
from search_engine import *
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from collections import defaultdict
import example_pos
from example_pos import *

In [70]:
#original index-contains just the abstract
# index_v2 contains the abstract introduction and conclusion
# 
f = open('index','rb')
index = pickle.load(f)
f.close()

query = 'review help suggest'
# Retrieve will return a ranked list of tuples of the form (documentID, documentScore).
print(Retrieve(query,index,1000,'nofeedback',0.1,10,50,0.9)[0:10])

[('P11-2088', 1.5825984651160894), ('N13-2011', 1.5782112598186024), ('W06-1650', 1.5023094397344914), ('W11-1402', 1.3557070868524779), ('W10-0207', 1.3074840527769733), ('W12-0608', 1.3059498797974205), ('J11-1014', 1.200329105921135), ('C10-2035', 1.1693846675458357), ('W10-1205', 1.1436679433762669), ('W13-5008', 1.0596568950756615)]


In [53]:
data = pickle.load(open('AllDataPickle_v2.pk','rb'))
len(data)

20399

In [69]:
stopwords = nltk.corpus.stopwords.words('english')+[',','.','!','``',"''",'?',"'s",';','$',':',"'",'_', ')', '(']
stemmer = PorterStemmer()
docCollection = defaultdict(list)
termFrequency = defaultdict(int)
idf = defaultdict(int)
docIndex = defaultdict(list)
tfidf = defaultdict(int)
numberofwords = 0
docnumberofwords = defaultdict(int)
numOfDocuments = 0
#TF-IDF scores of the documents
for key in data:
    future_work_section = data[key][-1]
    if not (future_work_section == ""):
        numOfDocuments = numOfDocuments + 1
        for token in [t.lower() for t in nltk.word_tokenize(future_work_section)]:
            if token in stopwords:
                continue
            if stemmer:
                token = stemmer.stem(token)
            if key not in docCollection:
                docCollection[key] = []

            docCollection[key].append(token)

            if token not in docIndex:
                docIndex[token] = defaultdict(int)

            if key not in docIndex[token]:
                docIndex[token][key] = 1

            else:
                docIndex[token][key] += 1

            numberofwords += 1
            docnumberofwords[key] += 1

            
for word in docIndex:
    sumword = 0
    for key in docIndex[word]:
        sumword = sumword + docIndex[word][key]
    termFrequency[word] = sumword/(numberofwords)
    idf[word] = 6*log ((numOfDocuments)/len(docIndex[word]), 10)
    tfidf[word] = termFrequency[word]*idf[word]

In [55]:
print (termFrequency['work'])
print (len(index['work']))
print (idf['work'])
print (tfidf['work'])


0.04051350257329555
3116
1.3821031645867
0.055993840115043186


In [56]:
[ item[0]  for item in sorted(termFrequency.items(),key=lambda k_v: k_v[1],reverse=True)][0:20]

['futur',
 'work',
 'use',
 'plan',
 'model',
 'improv',
 'system',
 'research',
 'method',
 'featur',
 'includ',
 'investig',
 'explor',
 'direct',
 'languag',
 'would',
 'data',
 'evalu',
 'approach',
 'perform']

In [57]:
sorted(idf.items(),key=lambda k_v: k_v[1],reverse=True)[0:100]

[('drosophila', 22.34368785858986),
 ('preorder', 22.34368785858986),
 ('preset', 22.34368785858986),
 ('varro', 22.34368785858986),
 ('xmg', 22.34368785858986),
 ('nonlinear', 22.34368785858986),
 ('hinrich', 22.34368785858986),
 ('geonod', 22.34368785858986),
 ('pelleg', 22.34368785858986),
 ('beeferman', 22.34368785858986),
 ('generr', 22.34368785858986),
 ('c99', 22.34368785858986),
 ('cognitive-sci', 22.34368785858986),
 ('selftrain', 22.34368785858986),
 ('mncd', 22.34368785858986),
 ('foltz', 22.34368785858986),
 ('irstlm', 22.34368785858986),
 ('fudg', 22.34368785858986),
 ('assur', 22.34368785858986),
 ('month', 22.34368785858986),
 ('slavon', 22.34368785858986),
 ('field-f1', 22.34368785858986),
 ('firmer', 22.34368785858986),
 ('imper', 22.34368785858986),
 ('olympu', 22.34368785858986),
 ('rm', 22.34368785858986),
 ('young', 22.34368785858986),
 ('autism', 22.34368785858986),
 ('197', 22.34368785858986),
 ('morpho', 22.34368785858986),
 ('6.1', 22.34368785858986),
 ('knob',

In [58]:
sorted(tfidf.items(),key=lambda k_v: k_v[1],reverse=True)

[('use', 0.05634966189308207),
 ('work', 0.055993840115043186),
 ('model', 0.05328370983671761),
 ('plan', 0.05117628462264162),
 ('system', 0.046978043216222155),
 ('improv', 0.04638533633326673),
 ('research', 0.04503347804454428),
 ('featur', 0.04497884952929483),
 ('method', 0.044916647250722926),
 ('includ', 0.042149627359092556),
 ('investig', 0.04166726209654704),
 ('languag', 0.03937286019702054),
 ('explor', 0.03888698963797192),
 ('direct', 0.03817588121160777),
 ('would', 0.03734931487894088),
 ('data', 0.03655577016290696),
 ('evalu', 0.03572463259325348),
 ('word', 0.034269393650078875),
 ('approach', 0.03399981056847884),
 ('inform', 0.03393584008538199),
 ('perform', 0.033409430705905775),
 ('differ', 0.03236077184847949),
 ('like', 0.032230032440843624),
 ('task', 0.03199021038915499),
 ('also', 0.030266760977505004),
 ('extend', 0.02994851696579758),
 ('gener', 0.02944606838597493),
 ('develop', 0.02917317746046707),
 ('result', 0.02908666244130895),
 ('semant', 0.0289

In [59]:
data_keys = data.keys()
fw_keys = []
num_FW = 0
last_nonKey = ""
for i, key in enumerate(data_keys):
    curr_FW = data[key][-1]
    if not (curr_FW == ""):
        print (i,curr_FW)
        num_FW = num_FW + 1
        fw_keys.append(key)
    else:
        last_nonKey = key
    

6 
we also would like to thank the reviewers for their helpful suggestions in directing the future of this work.
7 
this remains for our future work.
10 
in the future, we will conduct an annotation experiment with real users to evaluate the usefulness of rationales in terms of clock time.
12 
such research is left for the future as a possible extension to the modularity and cross-linguistic aspect of nlpwin.
15 
however, this is still an open issue and will have to be addressed in future research.
although such a detailed annotation of coherence relations was beyond the scope of the current project, future research should address this issue.
another issue that should be addressed in future research is empirically viable constraints on inferences for building discourse structures.
future research should investigate questions such as whether there are structural constraints on coherence graphs (e.g., as proposed by danlos [2004]) or whether there are systematic structural differences be

5296


In [60]:
def get_score(data, results):
    dat_score = 0
    t_c = 2014
    rank = 0
    denom = 0
    topK = 10
    avg_year = 0
    avg_score = 0
    for (key, rel_score) in results[0:topK]:
        rank = rank + 1;
        if (rank == 1):
            denom = rel_score
        t_d = data[key][2]
        if (t_c == t_d) or (rel_score/denom < 0.01):
            print (key, rel_score/denom)
        dat_score = dat_score + exp(t_d - t_c)*(rel_score/denom)
        avg_score = rel_score/denom
        avg_year = avg_year + t_d
        #print(key + " " + str(t_d))
    print (avg_year/topK, avg_score/topK)
    return dat_score

In [72]:
lta_scores = []
indicators = ['futur', 'work', 'use', 'plan', 'model', 'improv', 'system', 'research', 'method', 'featur', 'includ', 'investig', 'explor', 'direct', 'languag', 'would', 'data', 'evalu', 'approach', 'perform']
uninformative_fw = 0
for key in fw_keys[:1]:
    print(key)
    future_work_section = data[key][-1]
    abstract_section = data[key][6]
    query = future_work_section 
    print (query)
    phrases = " ".join(example_pos.get_phrases(query))
    clean_query = ""
    for item in phrases.split():
        if not( item in indicators):
            clean_query = clean_query + " " + item
    print (clean_query.strip())
    if not( clean_query.strip() == ""):
        result = Retrieve(clean_query.strip(),index,1000,'nofeedback',0.1,10,50,0.9)# [0:10]
        lta_scores.append( (key,get_score(data,result) )  )
    else:
        uninformative_fw = uninformative_fw + 1

W07-0204

we also would like to thank the reviewers for their helpful suggestions in directing the future of this work.
review
help
help suggest
futur
work
review help suggest
2010.7 0.06695677510327497


In [51]:
#Debug
data['N13-2011']

['Helpfulness-Guided Review Summarization',
 ['Xiong, Wenting'],
 2013,
 'NAACL',
 ['C08-1103',
  'E09-1059',
  'P08-1031',
  'P11-2088',
  'D09-1032',
  'W12-3708',
  'C10-1039',
  'H05-1043',
  'N10-1122',
  'P11-1150',
  'P08-1036',
  'W06-1650',
  'E06-1039',
  'P12-1036'],
 ['CIn1', 'CIn2'],
 '\nreview mining and summarization has been a hot topic for the past decade.\na lot of effort has been devoted to aspect detection and sentiment analysis under the assumption that every review has the same utility for related tasks.\nhowever, reviews are not equally helpful as indicated by user-provided helpfulness assessment associated with the reviews.\nin this thesis, we propose a novel review summarization framework which summarizes review content under the supervision of automated assessment of review helpfulness.\nthis helpfulness-guided framework can be easily adapted to traditional review summarization tasks, for a wide range of domains.',
 '\n2 related work  the proposed work is grou

In [41]:
sorted_scores = sorted(lta_scores, key=lambda x: x[1],reverse = True)

In [42]:
sorted_scores[0:10]

[('W07-0204', 0.7376442998215295),
 ('P09-2022', 0.3660613894842234),
 ('W12-2202', 0.3126140511356317),
 ('N09-3010', 0.12200302618107894),
 ('J05-2005', 0.10324274397827554),
 ('P09-1050', 0.03540509724253744),
 ('P08-3001', 0.024241383071877416),
 ('W02-1510', 0.022534243176556785),
 ('I05-3018', 0.021915779847156317)]

In [68]:
data['P11-2088']

['Automatically Predicting Peer-Review Helpfulness',
 ['Xiong, Wenting', ' Litman, Diane J.'],
 2011,
 'Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies',
 ['W06-1650', 'W09-3605', 'P05-1012', 'C00-1072'],
 ['N13-2011', 'W12-2020', 'W13-5008', 'W11-1402'],
 '\nidentifying peer-review helpfulness is an important task for improving the quality of feedback that students receive from their peers.\nas a first step towards enhancing existing peerreview systems with new functionality based on helpfulness detection, we examine whether standard product review analysis techniques also apply to our new context of peer reviews.\nin addition, we investigate the utility of incorporating additional specialized features tailored to peer review.\nour preliminary results show that the structural features, review unigrams and meta-data combined are useful in modeling the helpfulness of both peer reviews and product reviews, while peer-re